In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
import textblob as tb
import gensim
import json
import re
from pprint import pprint
import os, pickle
import ast
import datetime
import math


In [75]:
from textblob import TextBlob as tb
from nltk.corpus import stopwords 

In [76]:
df=pd.read_csv("reddit.csv")


In [146]:
len(df2)

4518

In [140]:
df2 = df.drop_duplicates("id")
df3= df2.sample(1000)

In [141]:
df3.head()

,author,body,comment,comms_num,created,id,score,timestamp,title,upvote,url,view_num
7754,Charlie_Yu,NaN,['This new Infernal Affairs movie looks much w...,265,1.563868e+09,cgl20c,1800,2019-07-23 15:53:06,The white shirt man who shaked hands with Juni...,NaN,https://i.redd.it/mqix492h0yb31.jpg,NaN
7054,Leaf-Currency,NaN,"['This is all kinds of amazing.', 'China fails...",194,1.570735e+09,dfwgk4,19828,2019-10-11 03:11:46,What a wild ride,NaN,https://i.redd.it/ey71kfji5pr31.jpg,NaN
5534,suprasa1yan,NaN,['What a son of a bitch. And Carrie Lam still...,18,1.571737e+09,dlak6m,183,2019-10-22 17:32:02,"HK Police swearing, instigating violence, call...",NaN,https://www.youtube.com/watch?v=SvvkS1yTRwo&fe...,NaN
5270,Sporeboss,NaN,['Twitter thread for this topic:\n\n\nhttps://...,8,1.571958e+09,dmhiym,122,2019-10-25 06:53:46,lab test result comparing both weapon use by h...,NaN,https://i.redd.it/y4qyc5jx5iu31.jpg,NaN
5296,ImANormalMan,Lets wish that they all win against the govern...,"[""I really hope you are right, and I hope that...",1,1.571936e+09,dmdkfw,24,2019-10-25 00:45:27,My wish for HongKong,NaN,https://www.reddit.com/r/HongKong/comments/dmd...,NaN


In [142]:
bloblist = []
i = 1
for com in df3[["comment"]].itertuples(index=False):
    for i in com : 
        
        
        try:
            bloblist.append(tb(i))
        except:
            pass

In [143]:
negativeWordSet = set(["don't","never", "nothing", "nowhere", "noone", "none", "not",
                  "hasn't","hadn't","can't","couldn't","shouldn't","won't",
                  "wouldn't","don't","doesn't","didn't","isn't","aren't","ain't","wasn't","weren't"])

stop_words = set(stopwords.words('english')) - set(negativeWordSet) 

stop_words.add('chu')

def tf_idf_process(blob):
    '''
    initially only .lower() & remove stopwords => find there are ♫, - etc non-alphabetic chr
        so we use .isalpha() 
    then we found words like "wk", "tm" have very high tf-idf values
        they are just initials of a person, no meaning
        so we use word.words() to check
    '''
    tag_dict = {"J": wordnet.ADJ, 
                "N": wordnet.NOUN, 
                "V": wordnet.VERB, 
                "R": wordnet.ADV}
   
    # lowercase + remove stopwords + remove non-alphabets + lemmatize 
    # english_words = set(words.words())
      
    processed_blob = [word.lower() for word in blob.words if word.isalpha()]
    lemmatized_blob = []
    for word, tag in nltk.pos_tag(processed_blob):
        if tag[0] in tag_dict:
            word = WordNetLemmatizer().lemmatize(word,tag_dict[tag[0]])
        lemmatized_blob.append(word)
    
    string = " ".join(lemmatized_blob)
    return tb(string)

def n_gram_top_phrases(n):

    n_gram_freq_dict = {}
    for blob in bloblist:
        words = blob.words
        for i in range(len(words)-(n-1)):
            word_phrase = tuple([words[i+x] for x in range(n)])
            in_stop_words = sum([word in stop_words for word in word_phrase]) != n
            
            if in_stop_words:
            
                if word_phrase in n_gram_freq_dict:
                    n_gram_freq_dict[word_phrase] += 1
                else:
                    n_gram_freq_dict[word_phrase] = 1
    #n_gram_freq_dict = clear_similar(n_gram_freq_dict)
    sorted_n_gram = sorted(n_gram_freq_dict.items(), key=lambda x: x[1], reverse=True)

    return n_gram_freq_dict

def check_similar(a,b):
    if len([x for x in a if x in b])>= len(a)-1:
        return True
    return False

def combine_similar(a,b):
    common = list(set(a) & set(b))
    output =[]
    if(len([x for x in a if x not in b]) >=1 or len([x for x in b if x not in a])>=1  ):
        if list(a).index(common[0]) > list(b).index(common[0]):
            output = list(a)
            output.extend([x for x in list(b) if x not in set(a)])
        else:
            output = list(b)
            output.extend([x for x in list(a) if x not in set(b)])

    return tuple(output)

def clear_similar(n_gram_freq_dict):
    no_similar_dict ={}
    for phrase1 in n_gram_freq_dict:
        similar = False
        for phrase2 in no_similar_dict:
            if check_similar(phrase1,phrase2):
                combined = combine_similar(phrase1,phrase2)
                no_similar_dict[combined] = max(no_similar_dict[phrase2],n_gram_freq_dict[phrase1])+1   
                del no_similar_dict[phrase2]
                similar = True
                break
        if not similar:
            no_similar_dict[phrase1] = n_gram_freq_dict[phrase1]
            
    return no_similar_dict

In [170]:
mafan= ["seen","'ve","wan","someone","like","look","got","would","kong","and","hong","'s",'https',"'s",'gon','www.youtube.com/watch','n',"s","they","see","i","in","a","lot","to","world","the","'m","do","i'","think","protestors","much","many","go","good","still","could","'m","'re","n\\n","also","world","this","say","going","they","i","s","not",'’',"n't","onw","the","get",'think',"it","even","know","see","really",'“','”']

In [171]:
msd = list(stop_words) + mafan

In [153]:
 trigram = n_gram_top_phrases(3)

In [155]:
sorted_t = sorted(trigram.items(), key=lambda kv: kv[1], reverse = True)

In [167]:
for element in sorted_t:
    i = element[0]
    count = element[1]
    
    if i[0].lower() not in msd and i[1].lower() not in msd and i[2].lower() not in msd and count > 10:
        
        print(i,count)

('one', 'country', 'two') 37
('Chinese', 'Communist', 'Party') 33
('country', 'two', 'systems') 33
('basic', 'human', 'right') 33
('human', 'rights', 'abuses') 32
('15', 'year', 'old') 28
('www.reddit.com/message/compose', 'to=kittens_from_space', 'Exclude') 28
('reddit.com/message/compose', 'to=WikiTextBot', 'message=Excludeme') 28
('to=WikiTextBot', 'message=Excludeme', 'subject=Excludeme') 28
('message=Excludeme', 'subject=Excludeme', 'Exclude') 28
('np.reddit.com/r/HongKong/about/banned', 'FAQ', 'Information') 28
('天安門', '天安门', '法輪功') 28
('天安门', '法輪功', '李洪志') 28
('Great', 'Leap', 'Forward') 26
('basic', 'human', 'rights') 26
('Tiananmen', 'Square', 'Massacre') 20
('Nobel', 'Peace', 'Prize') 20
('utm_source=share', 'utm_medium=ios_app', 'utm_name=iossmf') 20
('Contact', '32', 'Developer') 19
('32', '32', 'Support') 19
('32', 'Support', '32') 19
('www.paypal.me/synapsensalat', '32', '32') 19
('32', '32', 'Github') 19
('5Etfw', '7Ctwcamp', '5Etweetembed') 18
('One', 'Country', 'Two') 

In [144]:
bigram = n_gram_top_phrases(2)
sorted_b = sorted(bigram.items(), key=lambda kv: kv[1], reverse = True)

In [172]:
for element in sorted_b:
    i = element[0]
    count = element[1]
    
    if i[0].lower() not in msd and i[1].lower() not in msd and count > 50:
        kw = i[0] +" " + i[1]
        print(kw,count)

human rights 286
Chinese government 220
tear gas 196
HK police 194
police brutality 155
police force 149
Chinese people 140
Carrie Lam 135
universal suffrage 121
extradition bill 118
Tiananmen Square 114
United States 113
5 demands 103
police officer 102
years ago 101
mainland China 100
police officers 100
year old 85
first place 82
Human Rights 82
social media 76
free speech 75
trade war 71
long time 69
HK people 68
peaceful protest 67
fake news 65
basic human 65
North Korea 63
police station 63
riot police 61
human right 61
make sure 60
HK government 60
mainland Chinese 59
one thing 58
social credit 57
Communist Party 57
one day 56
concentration camps 56
every day 55
innocent people 55
3A 2F 55
million people 55
32 32 54
one country 52
metal rod 52
five demands 51
